Helper Functions

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
import pyamg
import random
import math
from torch.autograd import Variable
from pyamg.multilevel import multilevel_solver
from pyamg.relaxation.smoothing import change_smoothers
from scipy.sparse import csr_matrix, coo_matrix, lil_matrix, isspmatrix_csr, SparseEfficiencyWarning
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

def diffusion_stencil_2d(epsilon=1.0, theta=0.0, type='FE'):
    eps = float(epsilon)  # for brevity
    theta = float(theta)

    C = np.cos(theta)
    S = np.sin(theta)
    CS = C*S
    CC = C**2
    SS = S**2

    if(type == 'FE'):
        a = (-1*eps - 1)*CC + (-1*eps - 1)*SS + (3*eps - 3)*CS
        b = (2*eps - 4)*CC + (-4*eps + 2)*SS
        c = (-1*eps - 1)*CC + (-1*eps - 1)*SS + (-3*eps + 3)*CS
        d = (-4*eps + 2)*CC + (2*eps - 4)*SS
        e = (8*eps + 8)*CC + (8*eps + 8)*SS

        stencil = np.array([[a, b, c],
                            [d, e, d],
                            [c, b, a]]) / 6.0
    elif type == 'FD':
        a = -0.5*(eps - 1)*CS
        b = -(eps*SS + CC)
        c = -a
        d = -(eps*CC + SS)
        e = 2.0*(eps + 1)

        stencil = np.array([[a+c, d-2*c, 2*c],
                            [b-2*c, e+4*c, b-2*c],
                            [2*c, d-2*c, a+c]])
    return stencil

def map_2_to_1(grid_size_x=8,grid_size_y=8,stencil_size=3):
    # maps 2D coordinates to the corresponding 1D coordinate in the matrix.
    k = np.zeros((grid_size_x, grid_size_y, stencil_size, stencil_size))
    M = np.reshape(np.arange(grid_size_x * grid_size_y), (grid_size_x, grid_size_y))
    M = np.concatenate([M, M], 0)
    M = np.concatenate([M, M], 1)
    for i in range(stencil_size):
        I = (i - stencil_size//2) % grid_size_x
        for j in range(stencil_size):
            J = (j - stencil_size//2) % grid_size_y
            k[:, :, i, j] = M[I:I + grid_size_x, J:J + grid_size_y]
    return k
def get_p_matrix_indices_one(grid_size):
    K = map_2_to_1(grid_size=grid_size)
    indices = []
    for ic in range(grid_size // 2):
        i = 2 * ic + 1
        for jc in range(grid_size // 2):
            j = 2 * jc + 1
            J = int(grid_size // 2 * jc + ic)
            for k in range(3):
                for m in range(3):
                    I = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)

def compute_stencil(A, grid_size_x,grid_size_y,stencil_size):
    indices = get_indices_compute_A_one(grid_size_x,grid_size_y,stencil_size)
    stencil = np.array(A[indices[:, 0], indices[:, 1]]).reshape((grid_size_x, grid_size_y, stencil_size, stencil_size))
    return stencil
def get_indices_compute_A_one(grid_size_x=8,grid_size_y=8,stencil_size=3):
    indices = []
    K = map_2_to_1(grid_size_x,grid_size_y,stencil_size)
    for i in range(grid_size_x):
        for j in range(grid_size_y):
            I = int(K[i, j, stencil_size//2, stencil_size//2])
            for k in range(stencil_size):
                for m in range(stencil_size):
                    J = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)
def compute_p2(P_stencil, grid_size):
    indexes = get_p_matrix_indices_one(grid_size)
    P = sp.sparse.csr_matrix(arg1=(P_stencil.reshape(-1), (indexes[:, 1], indexes[:, 0])),
                   shape=((grid_size//2) ** 2, (grid_size) ** 2))

    return P
def compute_A_indices(grid_size_x=8,grid_size_y=8,stencil_size=3):
    K = map_2_to_1(grid_size_x,grid_size_y,stencil_size)
    A_idx = []
    stencil_idx = []
    for i in range(grid_size_x):
        for j in range(grid_size_y):
            I = int(K[i, j, stencil_size//2, stencil_size//2])
            for k in range(stencil_size):
                for m in range(stencil_size):
                    J = int(K[i, j, k, m])
                    A_idx.append([I, J])
                    stencil_idx.append([i, j, k, m])
    return np.array(A_idx), stencil_idx
def compute_p2_old(P_stencil, grid_size):
    indexes = get_p_matrix_indices_one_old(grid_size)
    P = sp.sparse.csr_matrix(arg1=(P_stencil.reshape(-1), (indexes[:, 1], indexes[:, 0])),
                   shape=((grid_size//2) ** 2, (grid_size) ** 2))

    return P
def get_p_matrix_indices_one_old(grid_size):
    K = map_2_to_1_old(grid_size=grid_size)
    indices = []
    for ic in range(grid_size // 2):
        i = 2 * ic + 1
        for jc in range(grid_size // 2):
            j = 2 * jc + 1
            J = int(grid_size // 2 * jc + ic)
            for k in range(3):
                for m in range(3):
                    I = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)
def map_2_to_1_old(grid_size=8):
    # maps 2D coordinates to the corresponding 1D coordinate in the matrix.
    k = np.zeros((grid_size, grid_size, 3, 3))
    M = np.reshape(np.arange(grid_size ** 2), (grid_size, grid_size))
    M = np.concatenate([M, M], 0)
    M = np.concatenate([M, M], 1)
    for i in range(3):
        I = (i - 1) % grid_size
        for j in range(3):
            J = (j - 1) % grid_size
            k[:, :, i, j] = M[I:I + grid_size, J:J + grid_size]
    return k

def prolongation_fn(grid_size):
#     grid_size = int(math.sqrt(A.shape[0]))
    res_stencil = np.double(np.zeros((3,3)))
    k=16
    res_stencil[0,0] = 1/k
    res_stencil[0,1] = 2/k
    res_stencil[0,2] = 1/k
    res_stencil[1,0] = 2/k
    res_stencil[1,1] = 4/k
    res_stencil[1,2] = 2/k
    res_stencil[2,0] = 1/k
    res_stencil[2,1] = 2/k
    res_stencil[2,2] = 1/k
    P_stencils= np.zeros((grid_size//2,grid_size//2,3,3))
    for i in range(grid_size//2):
        for j in range(grid_size//2):
            P_stencils[i,j,:,:]=res_stencil
    return compute_p2_old(P_stencils, grid_size).astype(np.double)  # imaginary part should be zero
def compute_A(P_stencil, grid_size_x,grid_size_y,stencil_size):
    A,indexes = compute_A_indices(grid_size_x,grid_size_y,stencil_size)
    P = torch.sparse.DoubleTensor(torch.LongTensor(A.T), P_stencil.view(-1), (grid_size_x*grid_size_y,grid_size_x*grid_size_y))
    return P
def compute_A_numpy(P_stencil, grid_size_x,grid_size_y,stencil_size):
    A,indexes = compute_A_indices(grid_size_x,grid_size_y,stencil_size)
    P = csr_matrix(arg1=(P_stencil.reshape((-1)), (A[:, 0], A[:, 1])),
                              shape=(grid_size_x*grid_size_y,grid_size_x*grid_size_y))
    return P

def coo_to_tensor(coo):
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.DoubleTensor(values)
    shape = coo.shape
    return torch.sparse.DoubleTensor(i, v, torch.Size(shape)).to(device)

def res_matrix(n):
    res_stencil = torch.zeros(3,3).double().to(device)
    k=4
    res_stencil[0,1] = 1/k
    res_stencil[1,1] = 2/k
    res_stencil[2,1] = 1/k
    R = pyamg.gallery.stencil_grid(res_stencil.cpu(), (n,n)).tocsr()
    idx=[]
    for i in range(n//2):
        idx=idx+list(range((i+1)*2*n-n,(i+1)*2*n))
    #R = torch.Tensor(R.toarray()).double()
    R = R[idx]
    return R

device: cpu


In [2]:
def coo_to_tensor(coo):
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.DoubleTensor(values)
    shape = coo.shape
    temp = coo
#     temp =sp.sparse.triu(coo,k=1)
#     temp = temp+temp.T
#     temp = temp.tocoo()
#     row = sp.sparse.triu(coo,k=1).row
#     col = sp.sparse.triu(coo,k=1).col
#     data = sp.sparse.triu(coo,k=1).data
#     for ii in range(len(row)):
#         if row[ii]==col[ii]:
#             data[ii]=data[ii]/2
    row = temp.row
    col = temp.col
    data = temp.data
    return torch.sparse_coo_tensor(i, v, torch.Size(shape),requires_grad = False)

In [3]:
A_train = []
s_train = []
stencil_train = []
eig_vec_train = []
Ag_train = []
A0_train = []
n = 15
# theta = np.pi/4
eps = 10
for iii in range(4):
    # theta = np.pi/6*np.random.rand(1).item()+np.pi/6
    theta = (iii+3)*np.pi/12
    # eps = 0.1+iii*0.01
    # theta = np.pi/4

    s = diffusion_stencil_2d(epsilon=eps, theta=theta, type='FD')
    A = pyamg.gallery.stencil_grid(s,(n,n)).tocsr()
    A0 = coo_to_tensor(A.tocoo())
    k=10
    # eig_value,eig_vec = sp.sparse.linalg.eigs(A,k=k,which = 'SM')
    R = prolongation_fn(n)
    P = R.T*4
    T = R*P

    A=R*A*P
    # print(eig_value)
    print(A.shape)

    Ag = coo_to_tensor(A.tocoo())
    eig_value,eig_vec = sp.sparse.linalg.eigs(A,k=k,M=T,which = 'SM')
    eig_vec = eig_vec.T

    eig_vec = torch.real(torch.from_numpy(eig_vec)).view(k,1,n//2,n//2)
    stencils = compute_stencil(A,n//2,n//2,3)
    s1 = stencils[n//4,n//4,:,:]
    AA_test = pyamg.gallery.stencil_grid(s1,(n//2,n//2))
    print(' err: ',np.linalg.norm(A.toarray()-AA_test.toarray()))
    A = nn.Conv2d(1, 1, 3, padding = 1, bias = False).double()
    A.weight = nn.Parameter(torch.from_numpy(s1).view(1,1,3,3),requires_grad = False)

    res_stencil = torch.zeros(1,1,3,3).double()
    res_stencil[0,0,0,0] = 1/16
    res_stencil[0,0,0,1] = 2/16
    res_stencil[0,0,0,2] = 1/16
    res_stencil[0,0,1,0] = 2/16
    res_stencil[0,0,1,1] = 4/16
    res_stencil[0,0,1,2] = 2/16
    res_stencil[0,0,2,0] = 1/16
    res_stencil[0,0,2,1] = 2/16
    res_stencil[0,0,2,2] = 1/16
    res = nn.Conv2d(1, 1, 3, padding = 0, stride=2, bias = False)
    print(res_stencil)
    res.weight = nn.Parameter(res_stencil)
    A_train.append(A)
    stencil_train.append(stencils)
    s_train.append(torch.from_numpy(s1))
    eig_vec_train.append(eig_vec)
    Ag_train.append(Ag)
    A0_train.append(A0)

(49, 49)
 err:  0.0
tensor([[[[0.0625, 0.1250, 0.0625],
          [0.1250, 0.2500, 0.1250],
          [0.0625, 0.1250, 0.0625]]]], dtype=torch.float64)
(49, 49)
 err:  0.0
tensor([[[[0.0625, 0.1250, 0.0625],
          [0.1250, 0.2500, 0.1250],
          [0.0625, 0.1250, 0.0625]]]], dtype=torch.float64)
(49, 49)
 err:  0.0
tensor([[[[0.0625, 0.1250, 0.0625],
          [0.1250, 0.2500, 0.1250],
          [0.0625, 0.1250, 0.0625]]]], dtype=torch.float64)
(49, 49)
 err:  0.0
tensor([[[[0.0625, 0.1250, 0.0625],
          [0.1250, 0.2500, 0.1250],
          [0.0625, 0.1250, 0.0625]]]], dtype=torch.float64)


In [4]:
def top_k(logits,k,tau):
    y_soft = logits.softmax(-1)
    index = torch.topk(y_soft,k)[1]
    y_hard = torch.zeros_like(logits, memory_format=torch.legacy_contiguous_format).scatter_(-1, index, 1.0)
    ret = y_hard - y_soft.detach() + y_soft

    return ret

In [5]:
class GNN_prob(nn.Module):
    def __init__(self):
        super(GNN_prob, self).__init__()
        self.fc1 = nn.Linear(8,500).double().to(device)
        self.fc2 = nn.Linear(500,8).double().to(device)

    def forward(self,X):
        X = self.fc1(X)
        # X = torch.relu(X)

        # X = torch.relu(X)
        X = torch.tanh(X)

        X = self.fc2(X)
        X = torch.tanh(X)

        return X
class GNN_value(nn.Module):
    def __init__(self):
        super(GNN_value, self).__init__()
        self.fc1 = nn.Linear(8,500).double().to(device)
        self.fc2 = nn.Linear(500,8).double().to(device)

    def forward(self,X):
        X = self.fc1(X)
        X = torch.tanh(X)
        # X = F.leaky_relu(X)

        X = self.fc2(X)
        # X = torch.relu(X)
        X = torch.tanh(X)
        # X = self.fc3(X)
        # # X = F.leaky_relu(X)
        # X = self.fc4(X)

        # X = F.leaky_relu(X)
        # X = self.fc5(X)

        # # X = torch.relu(X)
        # X = torch.tanh(X)
        # X = self.fc6(X)
        # # X = torch.relu(X)
        # X = torch.tanh(X)
        # X = F.leaky_relu(X)

        return X

def sparsify(prob,value,stencils):
    # prob = torch.sigmoid(X[0:8])
    # prob = X[0:8]
    n = stencils.shape[0]
    stencil = top_k(prob,4,0).squeeze()
    stencil = stencil*value
    # print(stencil)
    stencil = torch.cat((stencil[0:4],-stencil.sum().view(1),stencil[4:8])).view(1,1,3,3)
    # u,v = torch.topk(torch.sigmoid(prob),4)
    # for i in range(2,stencils.shape[0]-2):
    #   for j in range(2,stencils.shape[1]-2):
        # stencils[i,j,:,:] = stencil
    return stencil#,stencils




def train(A_train,stencil_train,s_train,eig_vec_train,epochs,Ag_train,A0_train):
    model_prob = GNN_prob()
    model_value = GNN_value()

#     model = GCN(32,32,10)
    optimizer = torch.optim.Adam(list(model_prob.parameters())+list(model_value.parameters()), lr=1e-3)
    tau = 1

    for epoch in range(epochs):
        loss = 0

        optimizer.zero_grad()
        for j in range(len(A_train)):
            A = A_train[j]
            stencils = torch.from_numpy(stencil_train[j])
            stencil0 = s_train[j]
            eig_vec = eig_vec_train[j]
            eig_vec = Variable(eig_vec.clone())
            A0 = A0_train[j]
            Ag = Ag_train[j]
            stencil = Variable(stencil0.clone().squeeze(0).squeeze(0)).reshape(9,1)
            stencil = torch.cat([stencil[0:4],stencil[5:9]]).t()
            orig_s = stencil0.clone().view(1,1,3,3)
            # r = 1e-1
            # if epoch%500==0:
            #     tau = max(0.05,np.exp(-r*epoch))
            tau = 0.05
            prob = model_prob(stencil).squeeze()
            value = model_value(stencil).squeeze()
            # value = stencil.squeeze()
            coarse_stencil= sparsify(prob,value,stencils)
            # A_c = compute_A(sstencils,stencils.shape[0],stencils.shape[1],5)

            temp = F.conv2d(eig_vec,coarse_stencil,padding=1)-A(eig_vec)
            # temp = torch.sparse.mm(A_c,eig_vec)-torch.sparse.mm(Ag,eig_vec)
            temp = temp.squeeze(1).view(temp.shape[0],-1,1)
            loss+=torch.norm(temp)**2
          # for i in range(temp.shape[0]):
          #   # loss+= torch.mm(temp[i,:].t(),torch.sparse.mm(Ag,torch.sparse.mm(Ag,temp[i,:])))
          #   # loss += torch.mm(temp[i,:].t(),torch.sparse.mm(Ag,temp[i,:]))

          #   # loss += torch.mm(temp[i,:].t(),torch.mm(torch.inverse(Ag.to_dense()),temp[i,:]))
          #   loss += torch.norm(temp[i,:])**2

            # loss+= torch.mm(temp[i,:].t(),torch.sparse.mm(Ag,temp[i,:]))

        # loss = (torch.norm(temp.squeeze(1),dim=(1,2))**2).mean()
        # n = n//2
        # A = project_stencil(coarse_stencil)
        # stencil = A.weight
            # orig_s = project_stencil(orig_s).weight
            # A = project_stencil(orig_s)
            # eig_vec = torch.rand(num_eigenvecs,1,n,n).double()
            # b = torch.rand(num_eigenvecs,1,n,n).double()
            # for k in range(10):
            #   # eig_vec = np.matmul(np.linalg.inv(L.toarray()),b-np.matmul(U.toarray(),eig_vec))
            #   eig_vec = 2/(3*coarse_stencil[0,0,1,1])*(b-A_c(eig_vec))+eig_vec
            # eig_vec = b-A_c(eig_vec)
            # eig_vec = res(eig_vec)
        loss.backward()
        # for name, param in model_value.named_parameters():
        #     print(name, param.grad)
        optimizer.step()

        if epoch%500==0:
            print(' epoch: ',epoch,' loss: ',loss)
    return model_prob,model_value

In [6]:
# n = 31
# num_eigenvecs = n*n
# eig_vec = torch.rand(num_eigenvecs,1,n,n).double()
# b = torch.rand(num_eigenvecs,1,n,n).double()
# for k in range(10):
#   # eig_vec = np.matmul(np.linalg.inv(L.toarray()),b-np.matmul(U.toarray(),eig_vec))
#   eig_vec = 2/(3*A0.weight[0,0,1,1])*(b-A0(eig_vec))+eig_vec
# eig_vec = b-A0(eig_vec)
# eig_vec = res(eig_vec)

model_prob,model_value = train(A_train,stencil_train,s_train,eig_vec_train,2000,Ag_train,A0_train)

 epoch:  0  loss:  tensor(82.3105, dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(24.6201, dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(30.1558, dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1500  loss:  tensor(23.8823, dtype=torch.float64, grad_fn=<AddBackward0>)


In [7]:
def geometric_solver(A,option1,option2,models,n,
                     presmoother=('gauss_seidel', {'sweep': 'forward'}),
                     postsmoother=('gauss_seidel', {'sweep': 'forward'}),
                     max_levels=5, max_coarse=10,coarse_solver='splu',stencil=0,**kwargs):

    levels = [multilevel_solver.level()]

    # convert A to csr
    if not isspmatrix_csr(A):
        try:
            A = csr_matrix(A)
            warn("Implicit conversion of A to CSR",
                 SparseEfficiencyWarning)
        except BaseException:
            raise TypeError('Argument A must have type csr_matrix, \
                             or be convertible to csr_matrix')
    # preprocess A
    A = A.asfptype()
    if A.shape[0] != A.shape[1]:
        raise ValueError('expected square matrix')

    levels[-1].A = A
    levels[-1].stencil = stencil
    levels[-1].n = n

    while len(levels) < max_levels and levels[-1].A.shape[0] > max_coarse:
        extend_hierarchy(levels,option1,option2,models,stencil)

    ml = multilevel_solver(levels, **kwargs)
    change_smoothers(ml, presmoother, postsmoother)
    return ml

# internal function
def extend_hierarchy(levels,option1,option2,models,stencil):
    """Extend the multigrid hierarchy."""

    A = levels[-1].A
    n = levels[-1].n
    # Generate the interpolation matrix that maps from the coarse-grid to the

#     R = prolongation_fn(size)
#     P = R.T.tocsr()*4
    R = prolongation_fn(n)
    P = R.T

    if option1=='standard':
        # Form next level through Galerkin product
        A=R*A*P
        # print(A)
        # eig_value,eig_vec = sp.sparse.linalg.eigs(A,feature_dims,which = 'SM')
        # X = torch.real(torch.from_numpy(eig_vec).to(device))
        n=n//2
        levels[-1].P = P  # prolongation operator
        levels[-1].R = R  # restriction operator

        levels.append(multilevel_solver.level())
        A = A.astype(np.float64)  # convert from complex numbers, should have A.imag==0
        levels[-1].A = A.tocsr()
        print('standard: ',A.count_nonzero())

    elif option1=='ru':
        if len(levels) == 1:
            n=n//2
            stencils = compute_stencil(R*A*P,n,n,3)
            s = stencils[n//2,n//2,:,:]
            levels[-1].P = P  # prolongation operator
            levels[-1].R = R  # restriction operator
            levels.append(multilevel_solver.level())
            stencil = s.reshape(9,1)
            stencil = torch.from_numpy(stencil)
            stencil = torch.cat([stencil[0:4],stencil[5:9]]).t()
            prob = model_prob(stencil).squeeze()
            value = model_value(stencil).squeeze()

            XX = sparsify(prob,value,torch.from_numpy(stencils))
            # stencils = torch.from_numpy(stencils)
            # A_c,stencil = ru(model,stencil,n)
            # A_c = pyamg.gallery.stencil_grid(XX.reshape(3,3).detach().numpy(),(n,n))
            # for i in range(2,n-2):
            #   for j in range(2,n-2):
            #     stencils[i,j,:,:] = XX
            # abc = compute_A_numpy(stencils.detach().numpy(),n,n,3)
            A_g = R*A*P
            A_c = pyamg.gallery.stencil_grid(XX.squeeze().detach().numpy(),(n,n))
            # print(XX)
            # I = np.identity(A_g.shape[0])
            # SSS = I-np.matmul(A_g.toarray(),np.linalg.inv(A_c.toarray()))
            # print('theta',np.linalg.norm(SSS,2))
            # print('???',np.linalg.norm(abc.toarray()-A_c.toarray()))
            # A_c = pyamg.gallery.stencil_grid(XX.detach().numpy().reshape(5,5),(n,n)).tocsr()
            levels[-1].A = A_c
            levels[-1].stencil = stencil
            print('ru: ',A_c.count_nonzero())
        else:
            A=R*A*P
            # print(A)
            # eig_value,eig_vec = sp.sparse.linalg.eigs(A,feature_dims,which = 'SM')
            # X = torch.real(torch.from_numpy(eig_vec).to(device))
            n=n//2
            levels[-1].P = P  # prolongation operator
            levels[-1].R = R  # restriction operator

            levels.append(multilevel_solver.level())
            A = A.astype(np.float64)  # convert from complex numbers, should have A.imag==0
            levels[-1].A = A.tocsr()
            print('ru: ',A.count_nonzero())

    else:
        levels[-1].P = P  # prolongation operator
        levels[-1].R = R  # restriction operator
        levels.append(multilevel_solver.level())

        n=n//2
        stencils = compute_stencil(R*A*P,n,n,3)
        s = stencils[n//2,n//2,:,:].reshape(-1,1)
        s = np.concatenate((s[0:4].reshape(-1,1),s[5:9].reshape(-1,1)))
        idx = np.argsort(s,axis=0)
        s[idx[0:4]]=0
        s = np.concatenate((s[0:4].reshape(-1,1),-s.sum().reshape(1,1),s[4:8].reshape(-1,1))).reshape(3,3)
        A_c = pyamg.gallery.stencil_grid(s,(n,n))
        # print('???',np.linalg.norm(abc.toarray()-A_c.toarray()))
        # A_c = pyamg.gallery.stencil_grid(XX.detach().numpy().reshape(5,5),(n,n)).tocsr()
        levels[-1].A = A_c
        levels[-1].stencil = stencil
#         print(A_c.shape)
        print('naive',A_c.count_nonzero())
    levels[-1].n=n


In [8]:
# torch.set_printoptions(precision=10)
num_test = 1
t1=time.time()
num_iter = []
num_iter2 = []
num_iter3 = []
num_iter4 = []
t_iter = []
t_iter2 = []
t_iter3 = []
t_iter4 = []
res_s = []
res2_s = []
res3_s = []
res4_s = []
test_grid_size = 127
model1=0
model2=0
model3=0
for i in range(num_test):
    # eps = 0.161+np.random.rand()*0.029
    eps = 10
    # theta = np.pi/6*np.random.rand(1).item()+np.pi/6
    # theta = (4+np.random.rand())*np.pi/12
    theta = np.pi/3
    s = diffusion_stencil_2d(epsilon=eps, theta=theta, type='FD')
    A_orig = pyamg.gallery.stencil_grid(s,(test_grid_size,test_grid_size)).tocsr()

    solver_standard = geometric_solver(A_orig,'standard',0,[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu')
    solver_non_galerkin = geometric_solver(A_orig,'ru','GNN',[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu', stencil = torch.from_numpy(s).reshape(1,1,3,3))
    # solver_naive = geometric_solver(A_orig,'SPSA','123',[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu',stencil = torch.from_numpy(s).reshape(1,1,3,3))

    x0 = np.ones((A_orig.shape[0],1))
    b = np.random.rand(A_orig.shape[0],1)

    res=[]
    res2= []
    res3= []
    res4 = []
    t1 =time.time()
    x = solver_standard.solve(b,x0=x0,maxiter=10, tol=1e-6,residuals=res)
    t2=time.time()
    x = solver_non_galerkin.solve(b,x0=x0,maxiter=10, tol=1e-6,residuals=res2)
    t3=time.time()
    # x = solver_naive.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res3)
    t4=time.time()
    # x = solver_naive.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res4)
    # t5=time.time()

    res_s.append(res)
    res2_s.append(res2)
    res3_s.append(res3)
    # res4_s.append(res4)

    num_iter.append(len(res))
    num_iter2.append(len(res2))
    num_iter3.append(len(res3))
    # num_iter4.append(len(res4))

    t_iter.append(t2-t1)
    t_iter2.append(t3-t2)
    t_iter3.append(t4-t3)
    # t_iter4.append(t5-t4)


print('standard iter:   ',np.mean(num_iter),'  standard time:    ',np.mean(t_iter))
print('Ru iter:   ',np.mean(num_iter2),'  Ru time:    ',np.mean(t_iter2))
# print('SPSA iter:   ',np.mean(num_iter3),'  SPSA time:    ',np.mean(t_iter3))
# print('naive iter:   ',np.mean(num_iter4),'  non galerkin time:    ',np.mean(t_iter4))


/var/folders/z1/ss2ybbnj0f796r9tcfgnd59w001pyt/T/ipykernel_46308/4116922730.py:6: DeprecationWarning: level() is deprecated.  use Level()
  levels = [multilevel_solver.level()]
/var/folders/z1/ss2ybbnj0f796r9tcfgnd59w001pyt/T/ipykernel_46308/4116922730.py:56: DeprecationWarning: level() is deprecated.  use Level()
  levels.append(multilevel_solver.level())
/var/folders/z1/ss2ybbnj0f796r9tcfgnd59w001pyt/T/ipykernel_46308/4116922730.py:29: DeprecationWarning: multilevel_solver is deprecated.  use MultilevelSolver()
  ml = multilevel_solver(levels, **kwargs)
/var/folders/z1/ss2ybbnj0f796r9tcfgnd59w001pyt/T/ipykernel_46308/4116922730.py:68: DeprecationWarning: level() is deprecated.  use Level()
  levels.append(multilevel_solver.level())


standard:  34969
ru:  19469
standard iter:    11.0   standard time:     31.291899919509888
Ru iter:    11.0   Ru time:     25.764727115631104


In [9]:
print(res)
print(res2)

[144.79259698861193, 32.50634653079042, 8.33755589820713, 2.936068701537597, 1.1301459579159765, 0.4471077197454943, 0.17905681034505025, 0.07226025879162125, 0.02932338983539053, 0.01195068361562142, 0.004887352220261888]
[144.79259698861193, 143.3003386355875, 153.4912049338964, 153.13989970937808, 149.59130125716243, 145.43363542555076, 141.18115710942416, 137.00923159890627, 133.00043432625196, 129.19150766390823, 125.5946271129056]
